# Client Manager là gì?
Trong Flower, client manager là một class trừu tượng nhằm điều hành các Flower Client. Đặc biệt tác vụ quan trọng nhất mà chúng ta có thể custom trong ClientManager là cách server chọn client để huấn luyện trên mỗi round. Một client manager cần phải kế thừa từ class ClientManager trong flower và có các hàm dưới đây:

In [ ]:
import flwr as fl
from flwr.server.client_manager import ClientManager
import threading
from abc import ABC, abstractmethod
from logging import INFO
from flwr.common.logger import log
from typing import Dict, List, Optional
import random
from flwr.server.client_proxy import ClientProxy
from flwr.server.criterion import Criterion

class SimpleClientManager(ClientManager):
    """Class clientManager đảm nhận việc xử lý các thông tin liên quan đến các client, như là việc chọn client cho mỗi round, vv"""

    def __init__(self) -> None:
        '''Khởi tạo các tham số cần thiết'''
    def __len__(self) -> int:
        """Trả về số client trong mô hình tổng """

    def num_available(self) -> int:
        """Trả về số client có thể tham gia vào huấn luyện"""

    def wait_for(self, num_clients: int, timeout: int = 86400) -> bool:
        """Cài đặt để chờ số client trước khi bắt đầu chọn cho round huấn luyện"""

    def register(self, client: ClientProxy) -> bool:

    def unregister(self, client: ClientProxy) -> None:

    def all(self) -> Dict[str, ClientProxy]:
        """Trả về tất cả các client có thể tham gia huấn luyện"""

    def sample(
        self,
        num_clients: int,
        min_num_clients: Optional[int] = None,
        criterion: Optional[Criterion] = None,
    ) -> List[ClientProxy]:
        """Chọn các client tham gia vào round huấn luyện"""

# Ví dụ
Đoạn code dưới đây được viết nhằm cố định các client được chọn ở mỗi round bằng cách thêm một giá trị seed vào

In [ ]:
class CustomClientManager(ClientManager):
    """Class clientManager đảm nhận việc xử lý các thông tin liên quan đến các client, như là việc chọn client cho mỗi round, vv"""

    def __init__(self) -> None:
        self.clients: Dict[str, ClientProxy] = {}
        self._cv = threading.Condition()
        self.seed = 0 # cài đặt seed để fix client tham gia mỗi round

    def __len__(self) -> int:
        """Trả về số client trong mô hình tổng
        """
        return len(self.clients)

    def num_available(self) -> int:
        """Trả về số client có thể tham gia vào huấn luyện
        """
        return len(self)

    def wait_for(self, num_clients: int, timeout: int = 86400) -> bool:
        """Cài đặt để chờ số client trước khi bắt đầu chọn cho round huấn luyện
        """
        with self._cv:
            return self._cv.wait_for(
                lambda: len(self.clients) >= num_clients, timeout=timeout
            )

    def register(self, client: ClientProxy) -> bool:
        if client.cid in self.clients:
            return False

        self.clients[client.cid] = client
        with self._cv:
            self._cv.notify_all()

        return True

    def unregister(self, client: ClientProxy) -> None:
        if client.cid in self.clients:
            del self.clients[client.cid]

            with self._cv:
                self._cv.notify_all()

    def all(self) -> Dict[str, ClientProxy]:
        """Trả về tất cả các client có thể tham gia huấn luyện"""
        return self.clients

    def sample(
        self,
        num_clients: int,
        min_num_clients: Optional[int] = None,
        criterion: Optional[Criterion] = None,
    ) -> List[ClientProxy]:
        """Chọn các client tham gia vào round huấn luyện"""
        # Block until at least num_clients are connected.
        random.seed(self.seed)
        self.seed += 1 # thay đổi giá trị seed để client được chọn ở mỗi round là khác nhau
        if min_num_clients is None:
            min_num_clients = num_clients
        self.wait_for(min_num_clients)
        # Sample clients which meet the criterion
        available_cids = list(self.clients)
        if criterion is not None:
            available_cids = [
                cid for cid in available_cids if criterion.select(self.clients[cid])
            ]

        sampled_cids = random.sample(available_cids, num_clients)
        print(sampled_cids)
        return [self.clients[cid] for cid in sampled_cids]